In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np

# 데이터 로드
# CSV 파일에서 데이터 로드
data = pd.read_csv('data.csv')

# 데이터 탐색
print("데이터 샘플:\n", data.head())
print("\n데이터 정보:\n")
data.info()
print("\n기술 통계:\n", data.describe())

# 결측치 확인
print("\n결측치 개수:\n", data.isnull().sum())

# 결측치 시각화
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(data.isnull(), cbar=False)
plt.show()

# 결측치 처리
from sklearn.impute import SimpleImputer

# 수치형 변수와 범주형 변수 구분
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()

# 수치형 변수 결측치 처리 (평균 대체)
mean_imputer = SimpleImputer(strategy='mean')
data[numerical_cols] = mean_imputer.fit_transform(data[numerical_cols])

# 범주형 변수 결측치 처리 (최빈값 대체)
mode_imputer = SimpleImputer(strategy='most_frequent')
data[categorical_cols] = mode_imputer.fit_transform(data[categorical_cols])

# 범주형 변수 인코딩 (원-핫 인코딩)
data = pd.get_dummies(data, columns=categorical_cols)

# 데이터 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# 최적의 클러스터 수 찾기 (엘보우 방법)
from sklearn.cluster import KMeans

wcss = []
k_range = range(1, 11)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_scaled)
    wcss.append(kmeans.inertia_)

# 엘보우 그래프 그리기
plt.figure(figsize=(8, 5))
plt.plot(k_range, wcss, 'bx-')
plt.xlabel('클러스터 수')
plt.ylabel('WCSS')
plt.title('엘보우 방법을 통한 최적의 클러스터 수 찾기')
plt.show()

# 최적의 클러스터 수 설정 (예: 3)
optimal_k = 3
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
clusters = kmeans.fit_predict(data_scaled)

# 클러스터링 결과 데이터프레임에 추가
data['Cluster'] = clusters

# 클러스터링 결과 시각화 (PCA 사용)
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_scaled)

plt.figure(figsize=(8, 5))
plt.scatter(data_pca[:, 0], data_pca[:, 1], c=clusters, cmap='viridis', edgecolor='k', s=50)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('클러스터링 결과 (PCA 변환된 데이터)')
plt.show()

# 클러스터링 평가 (실루엣 점수)
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(data_scaled, clusters)
print(f"\n실루엣 점수: {silhouette_avg:.4f}")